In [1]:
from pathlib import Path

In [2]:
from folder_sync import MetadataHasher
import shutil

mh = MetadataHasher()

p = Path.cwd().parent / "tests" / "test_data" / "test.txt"
print(p.stat())
print(mh(p))

p_copied = p.parent / "test2.txt"
shutil.copy2(p, p_copied)
print(p_copied.stat())
print(mh(p_copied))

os.stat_result(st_mode=33206, st_ino=281474977303970, st_dev=397746944, st_nlink=1, st_uid=0, st_gid=0, st_size=1, st_atime=1683363169, st_mtime=1683360778, st_ctime=1683360246)
test.txt_1683360778.8949096_1
os.stat_result(st_mode=33206, st_ino=281474977303973, st_dev=397746944, st_nlink=1, st_uid=0, st_gid=0, st_size=1, st_atime=1683363214, st_mtime=1683360778, st_ctime=1683360784)
test2.txt_1683360778.8949096_1


In [3]:
from uuid import uuid4

s1 = set([uuid4() for _ in range(1_000_000)])
s2 = set([uuid4() for _ in range(500_000)])
s = s1.union(s2)

%timeit s - s2
%timeit s - s1 

KeyboardInterrupt: 

In [ ]:
import stat

p = Path.cwd().parent / "tests" / "test_data" / "test.txt"

print(p.stat().st_mode)
print(stat.S_IFMT(p.stat().st_mode))

print(stat.S_IFREG)

In [ ]:
import filecmp

p1 = Path.cwd().parent / "tests" / "test_data" / "semester_10"
p2 = Path.cwd().parent / "tests" / "test_data" / "semester_10_copy"

# filecmp.dircmp(p1, p2).report_partial_closure()
# filecmp.dircmp(p1, p2).report_full_closure()
filecmp.dircmp(p1, p2).subdirs

In [ ]:
stats = {}

def sync_folder(source, target):
    for elem in target.iterdir():
        if elem.is_file():
            if elem not in source.iterdir():
                elem.unlink()
                stats["deleted_files"] += 1
        else:
            if elem not in source.iterdir():
                shutil.rmtree(elem)
                stats["deleted_folders"] += 1

    for elem in source.iterdir():
        if elem.is_file():
            if elem not in target.iterdir():
                shutil.copy2(elem, target)
                stats["copied_files"] += 1
            else:
                if hasher(elem) != hasher(target / elem.name):
                    shutil.copy2(elem, target)
                    stats["changed_files"] += 1
                else:
                    stats["unchanged_files"] += 1
        else: # is dir
            if elem not in target.iterdir():
                shutil.copytree(elem, target / elem.name)
            else:
                sync_folder(elem, target / elem.name)